In [ ]:
import numpy as np
import polars as pl

from sentence_transformers import SentenceTransformer
import torch

In [2]:
print(f"Доступно GPU: {torch.cuda.device_count()}")
device = 0 if torch.cuda.is_available() else -1
device

Доступно GPU: 1


0

In [ ]:
extractor = SentenceTransformer(
    model_name_or_path="Alibaba-NLP/gte-modernbert-base",
    device=device,
    model_kwargs=dict(
        torch_dtype=torch.float16,
        attn_implementation="flash_attention_2"
    )
)

In [ ]:
lazy_df = pl.scan_parquet("../../data/preprocessed/articles.parquet")
total = lazy_df.select(pl.len()).collect().item()
threshold = 100_000 / total

lazy_df = lazy_df.with_columns(
    pl.struct([pl.col("assets")]).map_elements(
        lambda _: np.random.rand(),
        skip_nulls=False,
        return_dtype=pl.Float64()
    ).alias("random")
)
sampled_lazy_df = (
    lazy_df
    .filter(pl.col("random") < threshold)
    .select(["datetime", "title", "source", "assets", "url", "text"])
)

data = sampled_lazy_df.collect().to_pandas()
texts = data.text.to_list()

In [ ]:
embeddings = extractor.encode(
    texts,
    show_progress_bar=True,
    normalize_embeddings=True
)

W0427 01:29:33.762000 126851 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


CPU times: user 2h 41min 47s, sys: 46.4 s, total: 2h 42min 33s
Wall time: 2h 36min 57s


In [ ]:
np.save("../aspects/data/embeddings_large.npy", embeddings)
data.to_csv("../aspects/data/data_large.csv", index=False)

In [7]:
torch.cuda.empty_cache()